In [1]:
import sys
sys.path.append('../')

In [2]:
from gymenv_v2 import make_multiple_env
import numpy as np

In [534]:
custom_config = {
    "load_dir"        : '../instances/randomip_n60_m60',   # this is the location of the randomly generated instances (you may specify a different directory)
    "idx_list"        : list(range(20)),                # take the first 20 instances from the directory
    "timelimit"       : 50,                             # the maximum horizon length is 50
    "reward_type"     : 'obj'                           # DO NOT CHANGE reward_type
}

# Easy Setup: Use the following environment settings. We will evaluate your agent with the same easy config below:
easy_config = {
    "load_dir"        : '../instances/train_10_n60_m60',
    "idx_list"        : list(range(10)),
    "timelimit"       : 50,
    "reward_type"     : 'obj'
}

# Hard Setup: Use the following environment settings. We will evaluate your agent with the same hard config below:
hard_config = {
    "load_dir"        : '../instances/train_100_n60_m60',
    "idx_list"        : list(range(99)),
    "timelimit"       : 50,
    "reward_type"     : 'obj'
}

s[0]: constraint matrix $A$of the current LP ($\max -c^Tx \text{ s.t. }Ax \le b$) . Dimension is $m \times n$. See by printing s[0].shape. Here $n$ is the (fixed) number of variables. For instances of size 60 by 60 used in the above command, $n$ will remain fixed as 60. And $m$ is the current number of constraints. Initially, $m$ is to the number of constraints in the IP instance. (For instances generated with --num-c=60, $m$ is 60 at the first step). But $m$ will increase by one in every step of the episode as one new constraint (cut) is added on taking an action.

s[1]: rhs $b$ for the current LP ($Ax\le b$). Dimension same as the number $m$ in matrix A.

s[2]: coefficient vector $c$ from the LP objective ($-c^Tx$). Dimension same as the number of variables, i.e., $n$.

s[3], s[4]: Gomory cuts available in the current round of Gomory's cutting plane algorithm. Each cut $i$ is of the form $D_i x\le d_i$. s[3] gives the matrix $D$ (of dimension $k \times n$) of cuts and s[4] gives the rhs $d$ (of dimension $k$). The number of cuts $k$ available in each round changes, you can find it out by printing the size of last component of state, i.e., s[4].size or s[-1].size.

In [536]:
env = make_multiple_env(**easy_config) 

loading training instances, dir ../instances/train_10_n60_m60 idx 0
loading training instances, dir ../instances/train_10_n60_m60 idx 1
loading training instances, dir ../instances/train_10_n60_m60 idx 2
loading training instances, dir ../instances/train_10_n60_m60 idx 3
loading training instances, dir ../instances/train_10_n60_m60 idx 4
loading training instances, dir ../instances/train_10_n60_m60 idx 5
loading training instances, dir ../instances/train_10_n60_m60 idx 6
loading training instances, dir ../instances/train_10_n60_m60 idx 7
loading training instances, dir ../instances/train_10_n60_m60 idx 8
loading training instances, dir ../instances/train_10_n60_m60 idx 9


In [ ]:
# s = env.reset()   # samples a random instance every time env.reset() is called
# d = False
# t = 0
# repisode = 0
# for e in range(20):
#     # gym loop
#     s = env.reset()   # samples a random instance every time env.reset() is called
#     d = False
#     t = 0
#     repisode = 0

#     while not d:
#         a = np.random.randint(0, s[-1].size, 1)            # s[-1].size shows the number of actions, i.e., cuts available at state s
#         s, r, d, _ = env.step(list(a))
#         print('episode', e, 'step', t, 'reward', r, 'action space size', s[-1].size, 'action', a[0])

#         A, b, c0, cuts_a, cuts_b = s
#         print(A.shape, b.shape, c0.shape, cuts_a.shape, cuts_b.shape)

#         t += 1
#         repisode += r

In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions.multivariate_normal import MultivariateNormal
use_cuda = torch.cuda.is_available()
print("Is CUDA available? %s.", use_cuda)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Is CUDA available? %s. False


In [6]:
from torch.nn.utils.rnn import pad_sequence

In [8]:
gamma = .8

In [120]:
s = env.reset()   # samples a random instance every time env.reset() is called
d = False
t = 0
repisode = 0


ABs = []
EDs = []

acts = []   # actions
rews = []  # instant rewards
obs = env.reset()
done = False

for _ in range(3):
    a = np.random.randint(0, s[-1].size, 1)
    ABs.append(preproc_ab(s[0], s[1]))
    EDs.append(preproc_ab(s[3], s[4]))
    
    s, r, d, _ = env.step(list(a))
    A, b, c0, cuts_a, cuts_b = s
    acts.append(s[-1])
    rews.append(r)

In [11]:
def discounted_rewards(r, gamma):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_sum = 0
    for i in reversed(range(0,len(r))):
        discounted_r[i] = running_sum * gamma + r[i]
        running_sum = discounted_r[i]
    return torch.FloatTensor(discounted_r)

In [714]:
class policyNet(nn.Module):
    def __init__(self, size_dim, hidden):
        super(policyNet, self).__init__()
        self.size_dim = size_dim
        self.fc1 = nn.Linear(size_dim, hidden)
        
    def forward(self, ab, ed):
        batch_size = ab.shape[0]
        h = F.relu(self.fc1(ab))
        g = F.relu(self.fc1(ed))
        g = torch.transpose(g, 1, 2)
        
        S = torch.mean(torch.bmm(h, g),1)
        return S
    

In [764]:
class policyNet(nn.Module):
    def __init__(self, size_dim, lstm_hidden):
        super(policyNet, self).__init__()
        self.size_dim = size_dim
        self.lstm_hidden = lstm_hidden
        self.LSTM = nn.LSTM(input_size=size_dim, 
                            hidden_size=lstm_hidden, 
                            batch_first=True)
        self.fc1 = nn.Linear(in_features=lstm_hidden,
                           out_features=size_dim)
        
    def forward(self, ab, ed):
        batch_size = ab.shape[0]
        ab, _ = self.LSTM(ab)
        h = F.relu(self.fc1(ab))
        ed, (hidden, _) = self.LSTM(ed)
        g = F.relu(self.fc1(ed))
        g = torch.transpose(g, 1, 2)
        S = torch.mean(torch.bmm(h, g),1)
        return S
    
    
class valueNetwork(nn.Module):
    def __init__(self, size_dim, lstm_hidden):
        super(valueNetwork, self).__init__()
        self.size_dim = size_dim
        self.lstm_hidden = lstm_hidden
        self.LSTM = nn.LSTM(input_size=size_dim, 
                            hidden_size=lstm_hidden, 
                            batch_first=True)
        self.fc1 = nn.Linear(in_features=lstm_hidden,
                           out_features=1)
        
    def forward(self, ab, ed):
        _, _ = self.LSTM(ab)
        _, (hidden, _) = self.LSTM(ed)
        
        hidden = F.relu(hidden.squeeze())
        output = self.fc1(hidden)
        
        return output.squeeze()

In [785]:
class policyNet(nn.Module):
    def __init__(self, size_dim, hidden):
        super(policyNet, self).__init__()
        self.size_dim = size_dim
        self.fc1 = nn.Linear(size_dim, hidden)
        
    def forward(self, ab, ed):
        batch_size = ab.shape[0]
        h = F.relu(self.fc1(ab))
        g = F.relu(self.fc1(ed))
        g = torch.transpose(g, 1, 2)
        
        S = torch.mean(torch.bmm(h, g),1)
        return S

class valueNetwork(nn.Module):
    def __init__(self, size_dim, hidden):
        super(valueNetwork, self).__init__()
        self.size_dim = size_dim
        self.fc1 = nn.Linear(size_dim, hidden)
        self.fc2 = nn.Linear(hidden, 1)
    def forward(self, ab, ed):
        batch_size = ab.shape[0]
        h = F.relu(self.fc1(ab))
        g = F.relu(self.fc1(ed))
        g = torch.transpose(g, 1, 2)
        J = torch.mean(torch.bmm(h, g),1)
        
        return J.mean(1)

In [806]:
model = valueNetwork(61, 64)
a = As[0]
b = Bs[0]
ab = torch.cat((a, b.unsqueeze(-1)), dim=-1)
e = Es[0]
d = Ds[0]
ed = torch.cat((e, d.unsqueeze(-1)), dim=-1)

In [808]:
ab.shape

torch.Size([50, 109, 61])

In [804]:
class ValueFunction(object):
    
    def __init__(self, size_dim, lstm_hidden, lr):
        
        self.model = valueNetwork(size_dim + 1, lstm_hidden)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=0.99)
        
    def compute_values(self, ab ,ed):
        with torch.no_grad():
            return self.model(ab, ed)
    
    def train(self, ab, ed, targets):
        
        v_preds = self.model(ab, ed)
        
        loss = torch.nn.functional.smooth_l1_loss(v_preds, targets)
        self.optimizer.zero_grad()
        loss.backward()
        # UPDATE
        self.optimizer.step()
        self.scheduler.step()
        return loss.item()
    
    

class Policy(object):
    
    def __init__(self, size_dim, lstm_hidden, lr):
        self.model = policyNet(size_dim + 1, lstm_hidden)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=0.99)
        
    def compute_prob(self, ab, ed):
        with torch.no_grad():
            prob = self.model(ab, ed)
            return prob
    
    def train(self, ab, ed, actions, Qs):
        logits = self.model(ab, ed)
        prob = torch.nn.functional.softmax(logits, dim=-1)
        
        prob_selected = prob.gather(-1, acts).squeeze()
        prob_selected += 1e-8
        
        loss = -torch.mean(Qs * torch.log(prob_selected))
        
        
        
        # BACKWARD PASS
        self.optimizer.zero_grad()
        loss.backward()
        # UPDATE
        self.optimizer.step()
        self.scheduler.step()
        return loss.item()  
    

In [805]:
def _preproc(s):
    min1 = min(s[0].min(), s[-2].min())
    max1 = max(s[0].max(), s[-2].max())
    min2 = min(s[1].min(), s[-1].min())
    max2 = max(s[1].max(), s[-1].max())
    
    A = torch.FloatTensor((s[0] - min1) / (max1 - min1))
    E = torch.FloatTensor((s[-2] - min1) / (max1 - min1))
    b = torch.FloatTensor((s[1] - min2) / (max2 - min2))
    d = torch.FloatTensor((s[-1] - min2) / (max2 - min2))
    
    return [A, b, torch.LongTensor(s[2]), E, d]

In [801]:
env = make_multiple_env(**easy_config)
N = 60


lstm_hidden_size = 64
alpha = 1e-2
beta = 1e-2
numtrajs = 5
iterations = 20
gamma = .8

policy = Policy(N, 64, alpha)
baseline = ValueFunction(N, 64, beta)

rrecord = []

for ite in range(iterations):
    
    
    t = 0
    repisode = 0
    
    As = []
    Bs = []
    Cs = []
    Es = []
    Ds = []
    
    
    ACTs = []
    VAL = []
    
    for num in range(numtrajs):
        
        obss = []
        acts = []
        rews = []
        
        s = env.reset()
        d = False

        while not d:
            ab, ed = _preproc_ab(s[0], s[1]), \
                     _preproc_ab(s[-2], s[-1])
            prob = policy.compute_prob(ab.unsqueeze(dim=0), ed.unsqueeze(dim=0)).squeeze()
            prob /= prob.sum()
            
            action = np.random.choice(s[-1].size, p = prob.numpy(), size=1)

            obss.append(_preproc(s))

            s, r, d, _ = env.step(list(action))
            A, b, c0, cuts_a, cuts_b = s
            acts.append(action)
            rews.append(r)
            
        rrecord.append(np.sum(rews))
        
        v_hat = discounted_rewards(rews, gamma)
        As.append(pad_sequence([elem[0] for elem in obss], batch_first=True))
        Bs.append(pad_sequence([elem[1] for elem in obss], batch_first=True))
        Cs.append(pad_sequence([elem[2] for elem in obss], batch_first=True))
        Es.append(pad_sequence([elem[-2] for elem in obss], batch_first=True))
        Ds.append(pad_sequence([elem[-1] for elem in obss], batch_first=True))
        
        
        ACTs.append(torch.tensor(acts))
        VAL.append(v_hat)
    

    for a, b, e, d, v in zip(As, Bs, Es, Ds, VAL):
        ab = torch.cat((a, b.unsqueeze(-1)), dim=-1)
        ed = torch.cat((e, d.unsqueeze(-1)), dim=-1)
        loss = baseline.train(ab, ed, v)
        
        
    baseline.optimizer.zero_grad()
    loss.backward()
    # UPDATE
    baseline.optimizer.step()
    baseline.scheduler.step()
        
        
    print(f'baseline loss: {loss.item()}')
    
    ADs = []
    
    for a, b, e, d, v in zip(As, Bs, Es, Ds, VAL):
        ab = torch.cat((a, b.unsqueeze(-1)), dim=-1)
        ed = torch.cat((e, d.unsqueeze(-1)), dim=-1)
        ADs.append(val - baseline.compute_values(ab, ed))
        
    for a, b, e, d, acts, ads in zip(As, Bs, Es, Ds, ACTs, ADs):
        ab = torch.cat((a, b.unsqueeze(-1)), dim=-1)
        ed = torch.cat((e, d.unsqueeze(-1)), dim=-1)
        loss = policy.train(ab, ed, acts, ads)
        
    policy.optimizer.zero_grad()
    loss.backward()
    # UPDATE
    policy.optimizer.step()
    policy.scheduler.step()
        
    print(f'agent loss: {loss}')

        
    print(f'iteration: {ite}, training reward: {np.mean(rrecord[-numtrajs:])}')
#     for ab_batch, ed_batch, v_hat in zip(ABs, EDs, VAL):
#         loss = baseline.train(ab_batch, ed_batch, v_hat)
#         print(f'baseline loss: {loss}')
#     ADs = []
#     for ab_batch, ed_batch, val in zip(ABs, EDs, VAL):
#         ADs.append(val - baseline.compute_values(ab_batch, ed_batch))          
#     for ab_batch, ed_batch, acts, ads in zip(ABs, EDs, ACTs, ADs):
#         loss = policy.train(ab_batch, ed_batch, acts, ads)
#         print(f'agent loss: {loss}')
        
#     print(f'iteration: {ite}, training reward: {rrecord[-1]}')

loading training instances, dir ../instances/train_10_n60_m60 idx 0
loading training instances, dir ../instances/train_10_n60_m60 idx 1
loading training instances, dir ../instances/train_10_n60_m60 idx 2
loading training instances, dir ../instances/train_10_n60_m60 idx 3
loading training instances, dir ../instances/train_10_n60_m60 idx 4
loading training instances, dir ../instances/train_10_n60_m60 idx 5
loading training instances, dir ../instances/train_10_n60_m60 idx 6
loading training instances, dir ../instances/train_10_n60_m60 idx 7
loading training instances, dir ../instances/train_10_n60_m60 idx 8
loading training instances, dir ../instances/train_10_n60_m60 idx 9
error in lp iteration


KeyboardInterrupt: 